In [ ]:
# import various libraries
import numpy as np
import pandas as pd
import sklearn
import pickle
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, KFold, cross_validate, train_test_split
from sklearn.inspection import permutation_importance
from sklearn import metrics
from sklearn.metrics import make_scorer, r2_score


In [ ]:
# directories and filenames
dir = '/content/drive/MyDrive/FateTrack/0716/' #directory where training/test data is saved
suff = '0716_allNuc_scaled_2-cy3_2-yfp_2-cy5_1-yfp_1-cy5_latent' #data suffix (usually date)
out = '0716_allNuc_scaled' #output suffix (usually same as suff)

In [ ]:
# import data
def get_dataset(final = False):
    X = pd.read_csv(dir + "nuclei_feat_" + suff + ".csv", header = None)
    sclr = StandardScaler()
    X = sclr.fit_transform(X)
    y = np.asarray(pd.read_csv(dir + "nuclei_meas_" + suff + ".csv", header = None))
    if final:
        filename = dir + out + '_final_scaler.pkl'
        pickle.dump(sclr, open(filename, 'wb'))
    return X, y

def get_dataset_split():
    X = pd.read_csv(dir + "nuclei_feat_" + suff + ".csv", header = None)
    sclr = StandardScaler()
    X = sclr.fit_transform(X)
    y = np.asarray(pd.read_csv(dir + "nuclei_meas_" + suff + ".csv", header = None))
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 2)
    return X_train, X_test, y_train, y_test

In [ ]:
# scorers
def clu_r2(y_true, y_pred):
  score = r2_score(y_true[:,0], y_pred[:,0])
  return score
clu_r2_score = make_scorer(clu_r2, greater_is_better=True)

def aldob_r2(y_true, y_pred):
  score = r2_score(y_true[:,1], y_pred[:,1])
  return score
aldob_r2_score = make_scorer(aldob_r2, greater_is_better=True)

def lyz1_r2(y_true, y_pred):
  score = r2_score(y_true[:,2], y_pred[:,2])
  return score
lyz1_r2_score = make_scorer(lyz1_r2, greater_is_better=True)

def muc2_r2(y_true, y_pred):
  score = r2_score(y_true[:,3], y_pred[:,3])
  return score
muc2_r2_score = make_scorer(muc2_r2, greater_is_better=True)

def chga_r2(y_true, y_pred):
  score = r2_score(y_true[:,4], y_pred[:,4])
  return score
chga_r2_score = make_scorer(chga_r2, greater_is_better=True)

In [ ]:
# 1) broad gridsearch
X, y = get_dataset()
model = RandomForestRegressor()
param_grid = {'n_estimators': np.arange(400, 1200, 200), 'max_features': np.arange(0.2, 1.2, 0.2), 'min_samples_leaf': [5, 10, 20], 'max_samples' = np.arange(0.2, 1.2, 0.2)}

scoring_dict = {'r2': make_scorer(r2_score), 'clu_r2': make_scorer(clu_r2), 'aldob_r2':make_scorer(aldob_r2),
                'lyz1_r2':make_scorer(lyz1_r2), 'muc2_r2':make_scorer(muc2_r2), 'chga_r2':make_scorer(chga_r2)}
                
grid_broad = GridSearchCV(model, param_grid, scoring = scoring_dict, verbose = 2, refit = 'r2')
grid_broad.fit(X, y)
n_estimators_best = grid_broad.best_params_['n_estimators']
max_features_best = grid_broad.best_params_['max_features']
min_samples_leaf_best = grid_broad.best_params_['min_samples_leaf']
max_samples_best = grid_broad.best_params_['max_samples']

res = pd.DataFrame(grid_broad.cv_results_)
res.to_csv(dir + "broadgridcv_" + out + ".csv")

In [ ]:
# 2) narrow gridsearch
X, y = get_dataset()
model = RandomForestRegressor()
param_grid = {'n_estimators': np.arange(n_estimators_best - 100, n_estimators_best + 150, 50), 
              'max_features': np.arange(max_features_best - 0.1, max_features_best + 0.15, 0.05), 
              'min_samples_leaf': range(min_samples_leaf_best - 4, min_samples_leaf_best + 6, 2), 
              'max_samples' = np.arange(max_samples_best - 0.1, max_samples_best + 0.15, 0.05)}

scoring_dict = {'r2': make_scorer(r2_score), 'clu_r2': make_scorer(clu_r2), 'aldob_r2':make_scorer(aldob_r2),
                'lyz1_r2':make_scorer(lyz1_r2), 'muc2_r2':make_scorer(muc2_r2), 'chga_r2':make_scorer(chga_r2)}

grid_narrow = GridSearchCV(model, param_grid, scoring = scoring_dict, verbose = 2, refit = 'r2')
grid_narrow.fit(X, y)
n_estimators_best = grid_narrow.best_params_['n_estimators']
max_features_best = grid_narrow.best_params_['max_features']
min_samples_leaf_best = grid_narrow.best_params_['min_samples_leaf']
max_samples_best = grid_narrow.best_params_['max_samples']

res = pd.DataFrame(grid_narrow.cv_results_)
res.to_csv(dir + "narrowgridcv_" + out + ".csv")

In [ ]:
# 3) k-fold cv
X, y = get_dataset()
model = RandomForestRegressor()
#model = RandomForestRegressor(n_estimators = n_estimators_best, max_features = max_features_best, min_samples_leaf = min_samples_leaf_best, max_samples = max_samples_best)

scoring_dict = {'r2': make_scorer(r2_score), 'clu_r2': make_scorer(clu_r2), 'aldob_r2':make_scorer(aldob_r2),
                'lyz1_r2':make_scorer(lyz1_r2), 'muc2_r2':make_scorer(muc2_r2), 'chga_r2':make_scorer(chga_r2)}

overall_r2_scores = []
clu_r2_scores = []
aldob_r2_scores = []
lyz1_r2_scores = []
muc2_r2_scores = []
chga_r2_scores = []
#pval = []
for rs in range(5):
  kf = KFold(n_splits = 5, shuffle = True, random_state = rs)

  scores = cross_validate(model, X, y, scoring = scoring_dict, cv = kf)
  overall_r2_scores.extend(scores["test_r2"])
  clu_r2_scores.extend(scores["test_clu_r2"])
  aldob_r2_scores.extend(scores["test_aldob_r2"])
  lyz1_r2_scores.extend(scores["test_lyz1_r2"])
  muc2_r2_scores.extend(scores["test_muc2_r2"])
  chga_r2_scores.extend(scores["test_chga_r2"])

clu_r2_mean = sum(clu_r2_scores) / len(clu_r2_scores)
aldob_r2_mean = sum(aldob_r2_scores) / len(aldob_r2_scores)
lyz1_r2_mean = sum(lyz1_r2_scores) / len(lyz1_r2_scores)
muc2_r2_mean = sum(muc2_r2_scores) / len(muc2_r2_scores)
chga_r2_mean = sum(chga_r2_scores) / len(chga_r2_scores)
# latent_r2_mean = sum(latent_r2_scores) / len(latent_r2_scores)
import statistics
clu_r2_stdev = statistics.pstdev(clu_r2_scores)
aldob_r2_stdev = statistics.pstdev(aldob_r2_scores)
lyz1_r2_stdev = statistics.pstdev(lyz1_r2_scores)
muc2_r2_stdev = statistics.pstdev(muc2_r2_scores)
chga_r2_stdev = statistics.pstdev(chga_r2_scores)
# latent_r2_stdev = statistics.pstdev(latent_r2_scores)

print(clu_r2_mean)
print(aldob_r2_mean)
print(lyz1_r2_mean)
print(muc2_r2_mean)
print(chga_r2_mean)
# print(latent_r2_mean)
print(clu_r2_stdev)
print(aldob_r2_stdev)
print(lyz1_r2_stdev)
print(muc2_r2_stdev)
print(chga_r2_stdev)

# clu_r2_scores = pd.DataFrame(clu_r2_scores)
# aldob_r2_scores = pd.DataFrame(aldob_r2_scores)
# lyz1_r2_scores = pd.DataFrame(lyz1_r2_scores)
# muc2_r2_scores = pd.DataFrame(muc2_r2_scores)
# chga_r2_scores = pd.DataFrame(chga_r2_scores)

# all_scores = pd.concat([clu_r2_scores, aldob_r2_scores, lyz1_r2_scores, muc2_r2_scores, chga_r2_scores], axis = 1)
# all_scores.to_csv(dir + "cv_scores" + out + ".csv")

0.46647145064543755
0.3537720071967401
0.024625966908226937
0.29845994628527484
0.10606844635723478
0.026775792698236208
0.05430186632078341
0.07110749859421091
0.04406985053138643
0.04255243843115209


In [ ]:
# 4) train_test_split to run the model 
X_train, X_test, y_train, y_test = get_dataset_split()
model = RandomForestRegressor(n_estimators = 500)
#model = RandomForestRegressor(n_estimators = n_estimators_best, max_features = max_features_best, min_samples_leaf = min_samples_leaf_best, max_samples = max_samples_best)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# export results
results_pred = pd.DataFrame(y_pred)
results_true = pd.DataFrame(y_test)
train = pd.DataFrame(y_train)
results_pred.to_csv(dir + "results_pred_" + out + ".csv")
results_true.to_csv(dir + "results_true_" + out + ".csv")
train.to_csv(dir + "train_" + out + ".csv")

perm_imp = permutation_importance(model, X_train, y_train)
perm_imp = pd.DataFrame(perm_imp.importances_mean)
perm_imp.to_csv(dir + "perm_importances_" + out + ".csv")

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multi

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 5) final model
X, y = get_dataset(final = True)
model = RandomForestRegressor(n_estimators = 500)
#model = RandomForestRegressor(n_estimators = n_estimators_best, max_features = max_features_best, min_samples_leaf = min_samples_leaf_best, max_samples = max_samples_best)
model.fit(X, y)
filename = dir + out + '_final_model.pkl'
pickle.dump(model, open(filename, 'wb'))